In [ ]:
import sys
import os

In [ ]:
ROOT_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(ROOT_PATH, "data")
MODELS_PATH = os.path.join(ROOT_PATH, "models")
YOLOV7_PATH = os.path.join(ROOT_PATH, "yolov7")
sys.path.append(YOLOV7_PATH)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import torch
import cv2
import numpy as np
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts


%matplotlib ipympl

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
weigths = torch.load(os.path.join(MODELS_PATH, 'yolov7.pt'))
model = weigths['model']
model = model.half().to(device)
_ = model.eval()

In [ ]:
model.yaml

In [ ]:
# Hack to torch 1.13.1
import torch.nn as nn

for m in model.modules():
    if isinstance(m, nn.Upsample):
        m.recompute_scale_factor = None

In [ ]:
left_test_image = cv2.imread(os.path.join(DATA_PATH, "keparoi_left_frame.jpg"))
right_test_image = cv2.imread(os.path.join(DATA_PATH, "keparoi_right_frame.jpg"))

In [ ]:
plt.imshow(left_test_image)
plt.show()

In [ ]:
%%time

def detect(orig_image):
    frame_width = orig_image.shape[1]
    frame_height = orig_image.shape[0]
    
    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
    image = letterbox(image, (frame_width), stride=64, auto=True)[0]
    image_ = image.copy()
    image = transforms.ToTensor()(image)
    image = torch.tensor(np.array([image.numpy()]))
    image = image.to(device)
    image = image.half()

    with torch.no_grad():
        output, _ = model(image)

    output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'])
    output = output_to_keypoint(output)
    #nimg = image[0].permute(1, 2, 0) * 255
    #nimg = nimg.cpu().numpy().astype(np.uint8)
    #nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
    
    n_players = 0
    
    for idx in range(output.shape[0]):

        class_id = output[idx][1]
        confidence = output[idx][6]

        if class_id != 0:
            continue

        if confidence < 0.7:
            continue

        #xmin, ymin = (output[idx, 2]-output[idx, 4]/2), (output[idx, 3]-output[idx, 5]/2)
        #max, ymax = (output[idx, 2]+output[idx, 4]/2), (output[idx, 3]+output[idx, 5]/2)
        #cv2.rectangle(
        # nimg,
        # (int(xmin), int(ymin)),
        # (int(xmax), int(ymax)),
        #  color=(255, 0, 0),
        #  thickness=1,
        #  lineType=cv2.LINE_AA
        #)
        
        n_players +=1
        
    print(f"Detected {n_players} players")

    #plt.imshow(nimg)
    #plt.show()

In [ ]:
%%time
detect(left_test_image)
detect(right_test_image)